In [25]:
#STT (speech to text)
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import pipeline
import librosa

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
# load model and processor for speech-to-text
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
modelw = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# modelw.config.forced_decoder_ids = None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [51]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [52]:
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en", device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
transcriber.device

device(type='cpu')

In [58]:
audio = "/Users/sasan.jafarnejad/dev/uni/talking-car/audio1713724521.779008.wav"
audio = "/Users/sasan.jafarnejad/dev/uni/talking-car/audio/attenborough/neutral.wav"
if type(audio) == str:
    link_to_audio = audio
    audio = torchaudio.load(link_to_audio)
    audio = audio[0].squeeze().numpy(), sr
    if len(audio[0].shape) == 2:
        audio = audio[0].mean(axis=0), audio[1]

In [59]:
y, sr = audio

In [60]:
out = transcriber({"sampling_rate": sr, "raw":y})["text"]

In [61]:
out

" If you speed up time, plants begin to reveal their true nature. They're not passive organisms, as you might think, but competitive creatures every bit as aggressive as animals. They're locked in a desperate battle for light and space. They stretch and pulse as they strive to barge their way into pole position. Creepers and vines reach around for the branch or stem of another plant on which to hitch a ride. This is an assassin bug. To us, it's easy enough to spot because it moves. To its prey, that's irrelevant, because it smells like one of their number. The assassin sucks its victims dry and blues their empty husks onto its back. This one is already carrying at least 20 corpses. Its irregular shape makes it hard for other predators to spot it and makes it virtually invisible to its prey, ants. It enters this ant colony unchallenged. Its coat of ant corpses masks its own odour. To the ants, it smells like one of their own, and that's what matters. They'll even run straight over the t

In [ ]:
def transcript(audio):
    if type(audio) == str:
        link_to_audio = audio
        audio = torchaudio.load(link_to_audio)

    # We assume that the audio is the audio tensor

    # process the audio array
    input_features = processor(audio_array, sampling_rate, return_tensors="pt").input_features
    predicted_ids = modelw.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    
    return audio_path, state['context'], state

In [ ]:
def transcript(general_context, link_to_audio, voice, place, time, delete_history, state):
    """this function manages speech-to-text to input Fnanswer function and text-to-speech with the Fnanswer output"""
    # load audio from a specific path
    audio_path = link_to_audio
    audio_array, sampling_rate = librosa.load(link_to_audio, sr=16000)  # "sr=16000" ensures that the sampling rate is as required

    # process the audio array
    input_features = processor(audio_array, sampling_rate, return_tensors="pt").input_features
    predicted_ids = modelw.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    


    return audio_path, state['context'], state

In [64]:
import numpy as np
import gradio as gr
import torchaudio
import time
import torch

def save_audio_as_wav(data, sample_rate, file_path):
    # make a tensor from the numpy array
    data = torch.tensor(data).reshape(1, -1)
    torchaudio.save(file_path, data, sample_rate=sample_rate, bits_per_sample=16, encoding="PCM_S")

def save_and_transcribe_audio(audio):
    # capture the audio and save it to a file as wav or mp3
    # file_name = save("audioinput.wav")
    sr, y = audio
    # y = y.astype(np.float32)
    # y /= np.max(np.abs(y))

    # add timestamp to file name
    filename = f"audio{time.time()}.wav"
    save_audio_as_wav(y, sr, filename)
    
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    text = transcriber({"sampling_rate": sr, "raw":y})["text"]
    return text

gr.Interface(
    fn=save_and_transcribe_audio, 
    inputs=gr.Audio(sources="microphone", type="numpy", label="Record Audio"), 
    outputs="text").launch()

Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
